## Libraries

In [1]:
import numpy as np 
import pandas as pd 
import os
from matplotlib import pyplot as plt
import keras as keras

In [2]:

from matplotlib import pyplot as plt
import cv2
data_dir = "../3244-2010-0025 Project/"
parasite = os.listdir('Parasitized/') 
uninfected = os.listdir('Uninfected/')
parasite.remove("Thumbs.db")               #databse file in both folders
uninfected.remove("Thumbs.db")
      

## Preprocessing and Augmentation

In [3]:
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

df_para, label_para = [], [] 



for filename in parasite:
    img = load_img(data_dir + "Parasitized/" + filename, target_size = (50, 50)) #Images resized to 50x50x3
    i = img_to_array(img)
    samples = expand_dims(i, 0)
    for j in range(2):
        datagen = ImageDataGenerator(rotation_range=360, fill_mode='nearest')   #Add 2 randomly rotated images from original
        aug_iter = datagen.flow(samples, batch_size=1)
        
        image = aug_iter.next()[0].astype('uint8')
        df_para.append(image)
    
    df_para.append(i)                          
    for i in range(3):
        label_para.append(1)        
    
df_un, label_un = [],[]            

for filename in uninfected:
    img = load_img(data_dir + "Uninfected/" + filename, target_size = (50, 50))
    i = img_to_array(img)
    samples = expand_dims(i, 0)
    for j in range(2):
        datagen = ImageDataGenerator(rotation_range=360, fill_mode='nearest')
        aug_iter = datagen.flow(samples, batch_size=1)
        
        image = aug_iter.next()[0].astype('uint8')
        df_un.append(image)
    df_un.append(i)                          
    for i in range(3):
        label_un.append(0)                            #label is a column of labels, 1 = parasitised, 0 = uninfected
    


In [ ]:
images = df_para + df_un
labels = label_para + label_un
images = np.array(images)
labels = np.array(labels)
images = images/255              #scaling the values so that they are between 0 and 1

In [ ]:
np.save("images",images)  #Saved the arrays to remove the need for reading imeages from files again
np.save("labels",labels)

### Load the training and test arrays

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras import optimizers
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import keras as keras
images = np.load("images.npy")
labels = np.load("labels.npy")

### Train Test Split
Splitting the data into training, validation and test sets. X_train_full is the complete training set to be used on test dataset. 

In [8]:
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score
np.random.seed(10)
X_train_full, X_test, y_train_full, y_test = sk.model_selection.train_test_split(images, labels, 
                                                                     test_size = 0.2, random_state = 10)

#Further split training dataset into training and validation
X_train, X_val, y_train, y_val = sk.model_selection.train_test_split(X_train_full, y_train_full, 
                                                                     test_size = 0.2, random_state = 10)

## Base Models
### Perceptron Learning Algorithm

In [ ]:
from sklearn.linear_model import Perceptron

clf = Perceptron(tol = None, max_iter = 1000,random_state=10,early_stopping=True,verbose=1)
mlp_X_train = np.array(X_train).reshape((-1,50*50*3))     #Reshape tensors to vectors 
mlp_y_train = np.array(y_train)
clf.fit(mlp_X_train,mlp_y_train)

### PLA Metrics

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, precision_score,recall_score
y_true = np.array(y_val)
y_pred = clf.predict(np.array(X_val).reshape((-1,7500)))
print(confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None))
print("Accuracy is ",accuracy_score(y_true, y_pred))
print("F1-Score is ",f1_score(y_true, y_pred))
print("Precision is ",precision_score(y_true, y_pred))
print("Recall is ",recall_score(y_true, y_pred))

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
LRmodel = LogisticRegression(random_state=10,max_iter=1000).fit(mlp_X_train, mlp_y_train)

### Logistic Regression Metrics

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, precision_score,recall_score
y_true = np.array(y_val)
y_pred = LRmodel.predict(np.array(X_val).reshape((-1,7500)))
print(confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None))
print("Accuracy is ",accuracy_score(y_true, y_pred))
print("F1-Score is ",f1_score(y_true, y_pred))
print("Precision is ",precision_score(y_true, y_pred))
print("Recall is ",recall_score(y_true, y_pred))

### Simplified VGG-16(Base)

In [ ]:
from keras import optimizers
basemodel = keras.Sequential([
    
    keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 3)),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(512, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
    
    ])
basemodel.compile( optimizer=optimizers.SGD(lr = 1e-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
basehistory = basemodel.fit(X_train, y_train,validation_data=(X_val,y_val), epochs=40, batch_size=32) 

### Base VGG-16 Metrics

In [ ]:
y_true = y_val
y_pred = basemodel.predict_classes(X_val)
print(confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None))
print("Accuracy is ",accuracy_score(y_true, y_pred))
print("F1-Score is ",f1_score(y_true, y_pred))
print("Precision is ",precision_score(y_true, y_pred))
print("Recall is ",recall_score(y_true, y_pred))

In [ ]:
cmbase = confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None)
print(cmbase[0][1]/(cmbase[0][1]+cmbase[0][0])) #FPR
print(cmbase[1][0]/(cmbase[1][0]+cmbase[1][1])) #FNR

## Hyperas Experiments
This section takes a very long time to run

In [ ]:
#Hyperas

#When running Hyperas we cant access global variables so need to save numpy arrays locally first
np.save('images.npy', images)
np.save('labels.npy', labels)

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

def data():
    import numpy as np
    import sklearn as sk
    from sklearn.model_selection import train_test_split
 
    images = np.load('images.npy')
    labels = np.load('labels.npy') 

    X_train, X_val, y_train, y_val = sk.model_selection.train_test_split(images, labels, test_size = 0.2, random_state = np.random)
    X_train = X_train/255
    X_val= X_val/255

    return X_train, y_train, X_val, y_val

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras import optimizers

def model(X_train, y_train, X_val, y_val):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (50, 50, 3)))
    model.add(MaxPooling2D(2, 2))
    
    model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(2, 2))
   
    model.add(Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(2, 2))

    model.add(Conv2D(256, (3, 3), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(2, 2))

    model.add(Conv2D(512, (3, 3), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(2, 2))
    
    model.add(Flatten())
    model.add(Dropout({{uniform(0, 1)}}))
   
    model.add(Dense({{choice([1024, 2048, 4096])}}, activation = 'relu'))
    model.add(Dense({{choice([1024, 2048, 4096])}}, activation = 'relu'))
    model.add(Dense(1, activation = {{choice(['sigmoid', 'softmax'])}})) #or model.add(Dense(2, activation = 'softmax'))

    adam = keras.optimizers.Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    rmsprop = keras.optimizers.RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
    sgd = keras.optimizers.SGD(lr={{choice([10**-3, 10**-2, 10**-1])}})
    adamax = keras.optimizers.Adamax(lr={{choice([10**-3, 10**-2, 10**-1])}})
    nadam = keras.optimizers.Nadam(lr={{choice([10**-3, 10**-2, 10**-1])}})
   
    choiceval = {{choice(['adam', 'rmsprop', 'sgd', 'adamax', 'nadam'])}}
    if choiceval == 'adam':
        optim = adam
    elif choiceval == 'rmsprop':
        optim = rmsprop
    elif choiceval == 'sgd':
        optim = sgd
    elif choiceval == 'adamax':
        optim = adamax
    else:
        optim = nadam
        
    model.compile(loss = 'binary_crossentropy', optimizer = optim, metrics = ['accuracy'])

    model.fit(X_train, y_train,
              batch_size = {{choice([16, 32, 50, 64])}},
              epochs= {{choice([10, 20, 50])}},
              verbose = 2,
              validation_data = (X_val, y_val))
    score, acc = model.evaluate(X_val, y_val, verbose = 0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

#print("Tuning hyperparams")

X_train, y_train, X_val, y_val = data()

best_run, best_model = optim.minimize(model = model,
                                      data = data,
                                      algo = tpe.suggest,
                                      max_evals = 30,
                                      trials = Trials())
print(best_run)

## Experiments on Optimisers
### RMSprop

In [ ]:
from keras import optimizers
model = keras.Sequential([
    
    keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 3)),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.17),
    
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
    
    ])
model.compile( optimizer=optimizers.RMSprop(lr = 1e-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
historyrms = model.fit(X_train, y_train,validation_data=(X_val,y_val), epochs=20, batch_size=64)
np.save('historyrms.npy',historyrms.history)

### Nadam

In [ ]:
from keras import optimizers
model = keras.Sequential([
    
    keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 3)),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.17),
    
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
    
    ])

model.compile( optimizer=optimizers.Nadam(lr = 1e-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
historynadam = model.fit(X_train, y_train,validation_data=(X_val,y_val), epochs=20, batch_size=64)
np.save('historynadam.npy',historynadam.history)

### Adamax

In [ ]:
from keras import optimizers
model = keras.Sequential([
    
    keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 3)),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.17),
    
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
    
    ])
#chg optimizer accordingly SGD,RMSprop,Adam,Adadelta,Adagrad,Adamax,Nadam,Ftrl

model.compile( optimizer=optimizers.Adamax(lr = 1e-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
historyadamax= model.fit(X_train, y_train,validation_data=(X_val,y_val), epochs=20, batch_size=64)
np.save('historyadamax.npy',historyadamax.history)

### Adam

In [ ]:
from keras import optimizers
model = keras.Sequential([
    
    keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 3)),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.17),
    
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
    
    ])
#chg optimizer accordingly SGD,RMSprop,Adam,Adadelta,Adagrad,Adamax,Nadam,Ftrl


model.compile( optimizer=optimizers.Adam(lr = 1e-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
historyadam= model.fit(X_train, y_train,validation_data=(X_val,y_val), epochs=20, batch_size=64)
np.save('historyadam.npy',historyadam.history)

### SGD

In [ ]:
from keras import optimizers
model = keras.Sequential([
    
    keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 3)),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.17),
    
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
    
    ])
#chg optimizer accordingly SGD,RMSprop,Adam,Adadelta,Adagrad,Adamax,Nadam,Ftrl

model.compile( optimizer=optimizers.SGD(lr = 1e-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
historysgd = model.fit(X_train, y_train,validation_data=(X_val,y_val), epochs=20, batch_size=64)
np.save('historysgd.npy',historysgd.history)

### Adadelta

In [ ]:
from keras import optimizers
model = keras.Sequential([
    
    keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 3)),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.17),
    
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
    
    ])
#chg optimizer accordingly SGD,RMSprop,Adam,Adadelta,Adagrad,Adamax,Nadam,Ftrl
model.compile( optimizer=optimizers.Adadelta(lr = 1e-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
historyadadelta = model.fit(X_train, y_train,validation_data=(X_val,y_val), epochs=20, batch_size=64)
np.save('historyadadelta.npy',historyadadelta.history)

## Plotting validation loss over epochs for each optimiser

In [ ]:
historysgd=np.load('historysgd.npy',allow_pickle='TRUE').item()              #Load the histories
historyrms=np.load('historyrms.npy',allow_pickle='TRUE').item()
historyadam=np.load('historyadam.npy',allow_pickle='TRUE').item()
historynadam=np.load('historynadam.npy',allow_pickle='TRUE').item()
historyadamax=np.load('historyadamax.npy',allow_pickle='TRUE').item()
historyadadelta=np.load('historyadadelta.npy',allow_pickle='TRUE').item()

In [ ]:
plt.figure(figsize=(5,4))
plt.title("val_loss")
plt.plot(historysgd['val_loss'],'-b',label="sgd")
plt.plot(historyrms['val_loss'],'-g',label="rms")
plt.plot(historyadam['val_loss'],'-r',label="adam")
plt.plot(historynadam['val_loss'],'-o',label="nadam")
plt.plot(historyadamax['val_loss'],'-y',label="adamax")
plt.plot(historyadadelta['val_loss'],'-p',label="adadelta")
plt.legend(fontsize=9)
plt.show()

## Effect of weights on FNR

In [ ]:
wt = 4 #Repeated for 2,4,6,8,10
from keras import optimizers
baseweightedmodel = keras.Sequential([
    
    keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 3)),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(512, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
    
    ])
baseweightedmodel.compile( optimizer=optimizers.SGD(lr = 1e-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
baseweightedmodel.fit(X_train, y_train,validation_data=(X_val,y_val), epochs=20, batch_size=32, class_weight={0:1,1:wt}) 

### Weighted Model Metrics

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, precision_score,recall_score
y_true = y_val
y_pred = baseweightedmodel.predict_classes(X_val)
print(confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None))
print("Accuracy is ",accuracy_score(y_true, y_pred))
print("F1-Score is ",f1_score(y_true, y_pred))
print("Precision is ",precision_score(y_true, y_pred))
print("Recall is ",recall_score(y_true, y_pred))
cmweightedbase = confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None)
#print(cmbase[0][1]/(cmbase[0][1]+cmbase[0][0])) #FPR
print(cmweightedbase[1][0]/(cmweightedbase[1][0]+cmweightedbase[1][1])) #FNR

## Effect of number of VGG-16 layers on accuracy
Here we try out the different layers of VGG-16 to see if they achieve better results
### 7 layer model

In [ ]:
#7 Layers VGG (Our simplified model)
model = Sequential([

    #1st stack
    Conv2D(32, (3, 3), padding='same', activation = 'relu', input_shape = (50, 50, 3)),
    #Conv2D(32, (3, 3), padding='same', activation= 'relu'), 
    MaxPooling2D(2, 2),

    #2nd stack
    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #3rd stack
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #4th stack
    Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #5th stack
    Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    
    Dense(128, activation = 'relu'),
    #Dense(128, activation = 'relu'), 
    Dense(1, activation = 'sigmoid')
    
    ])
    
model.compile( optimizer=optimizers.RMSprop(lr = 10**-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
history = model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=20, batch_size=64)

In [ ]:
y_true = y_val
y_pred = model.predict_classes(X_val)
print(confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None))
print("Accuracy is ",accuracy_score(y_true, y_pred))
print("F1-Score is ",f1_score(y_true, y_pred))
print("Precision is ",precision_score(y_true, y_pred))
print("Recall is ",recall_score(y_true, y_pred))
cmbase = confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None)
print(cmbase[0][1]/(cmbase[0][1]+cmbase[0][0])) #FPR
print(cmbase[1][0]/(cmbase[1][0]+cmbase[1][1])) #FNR

### 12 layer model

In [ ]:
#12 Layers VGG
model = Sequential([

    #1st stack
    Conv2D(32, (3, 3), padding='same', activation = 'relu', input_shape = (50, 50, 3)),
    Conv2D(32, (3, 3), padding='same', activation= 'relu'), 
    MaxPooling2D(2, 2),

    #2nd stack
    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #3rd stack
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #4th stack
    Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #5th stack
    Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    
    Dense(128, activation = 'relu'),
    #Dense(128, activation = 'relu'), 
    Dense(1, activation = 'sigmoid')
    
    ])
    
model.compile( optimizer=optimizers.RMSprop(lr = 10**-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
history = model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=20, batch_size=64)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, precision_score,recall_score
y_true = y_val
y_pred = model.predict_classes(X_val)
print(confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None))
print("Accuracy is ",accuracy_score(y_true, y_pred))
print("F1-Score is ",f1_score(y_true, y_pred))
print("Precision is ",precision_score(y_true, y_pred))
print("Recall is ",recall_score(y_true, y_pred))
cmbase = confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None)
print(cmbase[0][1]/(cmbase[0][1]+cmbase[0][0])) #FPR
print(cmbase[1][0]/(cmbase[1][0]+cmbase[1][1])) #FNR

### 16 layer (Full VGG-16) model

In [ ]:
#16 Layers VGG (Full model)
model = Sequential([

    #1st stack
    Conv2D(32, (3, 3), padding='same', activation = 'relu', input_shape = (50, 50, 3)),
    Conv2D(32, (3, 3), padding='same', activation= 'relu'), 
    MaxPooling2D(2, 2),

    #2nd stack
    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #3rd stack
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #4th stack
    Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #5th stack
    Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dropout(0.5),
    
    Dense(128, activation = 'relu'),
    Dense(128, activation = 'relu'), 
    Dense(1, activation = 'sigmoid')
    
    ])
    
model.compile( optimizer=optimizers.RMSprop(lr = 10**-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
history = model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=20, batch_size=64)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, precision_score,recall_score
y_true = y_val
y_pred = model.predict_classes(X_val)
print(confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None))
print("Accuracy is ",accuracy_score(y_true, y_pred))
print("F1-Score is ",f1_score(y_true, y_pred))
print("Precision is ",precision_score(y_true, y_pred))
print("Recall is ",recall_score(y_true, y_pred))
cmbase = confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None)
print(cmbase[0][1]/(cmbase[0][1]+cmbase[0][0])) #FPR
print(cmbase[1][0]/(cmbase[1][0]+cmbase[1][1])) #FNR

## Transfer Learning with pre-trained VGG-16

In [ ]:
#This is using Keras' VGG16 architecture via transfer learning

from keras.applications.vgg16 import VGG16
from keras.models import Model

base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (50, 50, 3))
base_model = Model(inputs = base_model.input, outputs = base_model.get_layer('block5_conv2').output)

base_model.trainable = False

x = base_model.output
x = MaxPooling2D()(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)

#Add a logistic layer 
predictions = Dense(1, activation = 'sigmoid', name = 'predictions')(x)

#This is the model we will train
model = Model(inputs = base_model.input, outputs = predictions)
model.compile(optimizer = optimizers.RMSprop(lr = 10**-3), loss = 'binary_crossentropy', metrics = ['accuracy','AUC'])
history = model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=20, batch_size=64)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, precision_score,recall_score
y_true = y_val
y_pred = np.around(model.predict(X_val))

print(confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None))
print("Accuracy is ",accuracy_score(y_true, y_pred))
print("F1-Score is ",f1_score(y_true, y_pred))
print("Precision is ",precision_score(y_true, y_pred))
print("Recall is ",recall_score(y_true, y_pred))
cmbase = confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None)
print(cmbase[0][1]/(cmbase[0][1]+cmbase[0][0])) #FPR
print(cmbase[1][0]/(cmbase[1][0]+cmbase[1][1])) #FNR

## Final Modified VGG-16 model
We run the final model as well as the base model with the full training set and test their performance on the test set. 

In [ ]:
finalmodel = keras.Sequential([

    #1st stack
    Conv2D(32, (3, 3), padding='same', activation = 'relu', input_shape = (50, 50, 3)),
    Conv2D(32, (3, 3), padding='same', activation= 'relu'), 
    MaxPooling2D(2, 2),

    #2nd stack
    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #3rd stack
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #4th stack
    Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),

    #5th stack
    Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    #Conv2D(512, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    #Dropout(0.5), #Can try without dropout
    
    Dense(128, activation = 'relu'),
    #Dense(128, activation = 'relu'), 
    Dense(1, activation = 'sigmoid') 
    
    ])

finalmodel.compile( optimizer=optimizers.SGD(lr = 1e-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
history = finalmodel.fit(X_train_full, y_train_full,validation_data=(X_val,y_val), epochs=40, batch_size=32,class_weight={0:1,1:4}) #weight = 4

In [ ]:
ypred = pd.DataFrame(finalmodel.predict_classes(X_test))
errordf = pd.concat([ypred,pd.DataFrame(y_test)],axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, precision_score,recall_score
y_true = y_test
y_pred = finalmodel.predict_classes(X_test)
finalcmbase= confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None)
print(confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None))
print("Accuracy is ",accuracy_score(y_true, y_pred))
print("F1-Score is ",f1_score(y_true, y_pred))
print("Precision is ",precision_score(y_true, y_pred))
print("Recall is ",recall_score(y_true, y_pred))
print(finalcmbase[0][1]/(finalcmbase[0][1]+finalcmbase[0][0])) #FPR
print(finalcmbase[1][0]/(finalcmbase[1][0]+finalcmbase[1][1])) #FNR


### Base model with weighted penalty

In [ ]:
from keras import optimizers
basemodel = keras.Sequential([
    
    keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 3)),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(512, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
    
    ])
basemodel.compile( optimizer=optimizers.SGD(lr = 1e-3),loss='binary_crossentropy', metrics=['accuracy','AUC'])
basehistory = basemodel.fit(X_train_full, y_train_full,validation_data=(X_val,y_val), epochs=40, batch_size=32,class_weight={0:1,1:4}) 

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, precision_score,recall_score
y_true = y_test
y_pred = basemodel.predict_classes(X_test)
cmbase= confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None)
print(confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None))
print("Accuracy is ",accuracy_score(y_true, y_pred))
print("F1-Score is ",f1_score(y_true, y_pred))
print("Precision is ",precision_score(y_true, y_pred))
print("Recall is ",recall_score(y_true, y_pred))
print(cmbase[0][1]/(cmbase[0][1]+cmbase[0][0])) #FPR
print(cmbase[1][0]/(cmbase[1][0]+cmbase[1][1])) #FNR

In [ ]:
ypred = pd.DataFrame(basemodel.predict_classes(X_test))
errordf = pd.concat([ypred,pd.DataFrame(y_test)],axis=1)

## Plotting the errors

In [ ]:
#This is to find out how off is the predicted labels from actual labels

import matplotlib.pyplot as pltpy

predictions = (finalmodel.predict(X_test) > 0.5).astype(int).flatten()
wrong_predictions = finalmodel.predict(X_test)[predictions != y_test]
wrong_pred_list = []
for i in range(wrong_predictions.shape[0]):
    wrong_pred_list.append(wrong_predictions[i][0])

fn_list_pre = []
fp_list_pre = []

for i in range(len(wrong_pred_list)):
        if wrong_pred_list[i] >= 0.5:
            fp_list_pre.append(wrong_pred_list[i])
        else:
            fn_list_pre.append(wrong_pred_list[i])

fn_list = []
fp_list = []
            
for i in range(len(fn_list_pre)):
    fn_list.append(0.5 - fn_list_pre[i])
    
for i in range(len(fp_list_pre)):
    fp_list.append(fp_list_pre[i] - 0.5)
    
pltpy.hist(fn_list, bins = 20)     #False negative distribution

In [ ]:
pltpy.hist(fp_list, bins = 20)  #False positive distribution

## Identifying False Negative and False Positives

In [ ]:
#False Negatives and False Positives of Final Model
ypredfinal = pd.DataFrame(finalmodel.predict_classes(X_test))
errordffinal = pd.concat([ypred,pd.DataFrame(y_test)],axis=1)
ypredbase = pd.DataFrame(basemodel.predict_classes(X_test))
errordfbase = pd.concat([ypred,pd.DataFrame(y_test)],axis=1)

In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
misclassified = errordffinal[errordffinal.iloc[:,0] != errordffinal.iloc[:,1]]
FP = misclassified[misclassified.iloc[:,0]==1]
FN = misclassified[misclassified.iloc[:,0]==0]

### False Positive for Final VGG-16

In [ ]:
plt.figure(figsize=(11,5))
for i in range(1,11):
    plt.subplot(2,5,i)
    plt.imshow(X_test[FP.index[i]])
        

### False Negatives for Final VGG-16

In [ ]:
plt.figure(figsize=(10,5))
for i in range(1,11):
    plt.subplot(2,5,i)
    plt.imshow(X_test[FN.index[i]])

### Misclassified Images for Logistic Regression

In [ ]:
#False negatives and false positives of LogReg
y_true = np.array(y_val)
y_pred = LRmodel.predict(np.array(X_val).reshape((-1,7500)))
ypred = pd.DataFrame(y_pred)
errordf = pd.concat([ypred,pd.DataFrame(y_true)],axis=1)

misclassified = errordf[errordf.iloc[:,0] != errordf.iloc[:,1]]
FP = misclassified[misclassified.iloc[:,0]==1]
FN = misclassified[misclassified.iloc[:,0]==0]

### False Positives for Logistic Regression

In [ ]:
#False Positives
plt.title("False Positives")
plt.figure(figsize=(10,5))
for i in range(1,11):
    plt.title()
    plt.subplot(2,5,i)
    plt.imshow(X_test.reshape((-1,50,50,3))[FP.index[i]])

### False Negatives for Logistic Regression

In [ ]:
#False Negatives
plt.title("False Negatives")
plt.figure(figsize=(10,5))
for i in range(1,11):
    plt.subplot(2,5,i)
    plt.imshow(X_test.reshape((-1,50,50,3))[FN.index[i]])

### Misclassified Images for PLA

In [ ]:
#False positives and negatives for PLA
y_true = np.array(y_val)
y_pred = clf.predict(np.array(X_val).reshape((-1,7500)))
ypred = pd.DataFrame(y_pred)
errordf = pd.concat([ypred,pd.DataFrame(y_true)],axis=1)

misclassified = errordf[errordf.iloc[:,0] != errordf.iloc[:,1]]
FP = misclassified[misclassified.iloc[:,0]==1]
FN = misclassified[misclassified.iloc[:,0]==0]

### False Positives for PLA

In [ ]:
#False Positives
plt.title("False Positives")
plt.figure(figsize=(10,5))
for i in range(1,11):
    plt.title()
    plt.subplot(2,5,i)
    plt.imshow(X_test.reshape((-1,50,50,3))[FP.index[i]])

### False Negatives for PLA

In [ ]:
#False Negatives
plt.title("False Negatives")
plt.figure(figsize=(10,5))
for i in range(1,11):
    plt.subplot(2,5,i)
    plt.imshow(X_test.reshape((-1,50,50,3))[FN.index[i]])